In [ ]:
!pip install numpy pandas scikit-learn torch torchvision
!pip install surprise


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

In [ ]:
# Step 2: Load dataset files
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
links = pd.read_csv('/content/links.csv')
tags = pd.read_csv('/content/tags.csv')

In [ ]:
print("Movies sample:")
print(movies.head())
print("\nRatings sample:")
print(ratings.head())

Movies sample:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings sample:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
# Step 3: Preprocess ratings data
# Encode user and item IDs to continuous indices (required for embeddings)
user_ids = ratings['userId'].unique()
item_ids = ratings['movieId'].unique()

In [ ]:
user2idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
item2idx = {item_id: idx for idx, item_id in enumerate(item_ids)}

In [ ]:
ratings['user_idx'] = ratings['userId'].map(user2idx)
ratings['item_idx'] = ratings['movieId'].map(item2idx)

In [ ]:
# Step 4: Split into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Define PyTorch Dataset class
class MovieRatingDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_idx'].values, dtype=torch.long)
        self.items = torch.tensor(df['item_idx'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]


In [ ]:
train_dataset = MovieRatingDataset(train_data)
test_dataset = MovieRatingDataset(test_data)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


In [ ]:
num_users = len(user2idx)
num_items = len(item2idx)
print(f"Number of users: {num_users}, Number of items: {num_items}")

Number of users: 610, Number of items: 9724


In [ ]:
# Step 6: Define Model 1: Multilayer Perceptron (MLP)
class MLP(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64):
        super(MLP, self).__init__()
        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)

        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, user_idx, item_idx):
        user_vector = self.user_emb(user_idx)
        item_vector = self.item_emb(item_idx)
        x = torch.cat([user_vector, item_vector], dim=-1)
        out = self.fc_layers(x)
        return out.squeeze()

In [ ]:
# Step 7: Define Model 2: Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, num_items, embedding_dim=64):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(num_items, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_items),
            nn.Sigmoid()  # Since ratings are between 0 and 5, we can normalize later
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out

In [ ]:
# Step 8: Training helper functions

def train_mlp(model, optimizer, criterion, dataloader):
    model.train()
    total_loss = 0
    for user, item, rating in dataloader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        preds = model(user, item)
        loss = criterion(preds, rating)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(rating)
    return total_loss / len(dataloader.dataset)

def evaluate_mlp(model, dataloader):
    model.eval()
    preds_list = []
    true_list = []
    with torch.no_grad():
        for user, item, rating in dataloader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            preds = model(user, item)
            preds_list.append(preds.cpu().numpy())
            true_list.append(rating.cpu().numpy())
    preds_all = np.concatenate(preds_list)
    true_all = np.concatenate(true_list)
    rmse = math.sqrt(mean_squared_error(true_all, preds_all))
    return rmse

def train_autoencoder(model, optimizer, criterion, train_matrix):
    model.train()
    optimizer.zero_grad()
    output = model(train_matrix)
    loss = criterion(output, train_matrix)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate_autoencoder(model, test_matrix):
    model.eval()
    with torch.no_grad():
        output = model(test_matrix)
        loss = nn.MSELoss()(output, test_matrix)
        rmse = torch.sqrt(loss)
    return rmse.item()

In [ ]:
# Step 9: Prepare user-item rating matrix for autoencoder
def create_user_item_matrix(df, num_users, num_items):
    matrix = np.zeros((num_users, num_items), dtype=np.float32)
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating / 5.0  # Normalize rating between 0 and 1
    return torch.tensor(matrix).to(device)

train_matrix = create_user_item_matrix(train_data, num_users, num_items)
test_matrix = create_user_item_matrix(test_data, num_users, num_items)


In [ ]:
# Step 10: Instantiate models, optimizers, and loss functions
mlp_model = MLP(num_users, num_items).to(device)
autoencoder_model = Autoencoder(num_items).to(device)

mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)
autoencoder_optimizer = torch.optim.Adam(autoencoder_model.parameters(), lr=0.001)

criterion = nn.MSELoss()


In [ ]:
# Step 11: Train both models and evaluate

num_epochs = 10

print("Training MLP model:")
for epoch in range(num_epochs):
    train_loss = train_mlp(mlp_model, mlp_optimizer, criterion, train_loader)
    rmse = evaluate_mlp(mlp_model, test_loader)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test RMSE: {rmse:.4f}")

print("\nTraining Autoencoder model:")
for epoch in range(num_epochs):
    train_loss = train_autoencoder(autoencoder_model, autoencoder_optimizer, criterion, train_matrix)
    rmse = evaluate_autoencoder(autoencoder_model, test_matrix)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test RMSE: {rmse:.4f}")


Training MLP model:
Epoch 1, Train Loss: 3.4212, Test RMSE: 1.0276
Epoch 2, Train Loss: 1.3009, Test RMSE: 0.9869
Epoch 3, Train Loss: 1.2036, Test RMSE: 0.9648
Epoch 4, Train Loss: 1.1418, Test RMSE: 0.9501
Epoch 5, Train Loss: 1.1063, Test RMSE: 0.9424
Epoch 6, Train Loss: 1.0696, Test RMSE: 0.9372
Epoch 7, Train Loss: 1.0369, Test RMSE: 0.9317
Epoch 8, Train Loss: 1.0218, Test RMSE: 0.9267
Epoch 9, Train Loss: 1.0056, Test RMSE: 0.9210
Epoch 10, Train Loss: 0.9870, Test RMSE: 0.9164

Training Autoencoder model:
Epoch 1, Train Loss: 0.2478, Test RMSE: 0.4983
Epoch 2, Train Loss: 0.2466, Test RMSE: 0.4971
Epoch 3, Train Loss: 0.2450, Test RMSE: 0.4957
Epoch 4, Train Loss: 0.2428, Test RMSE: 0.4941
Epoch 5, Train Loss: 0.2398, Test RMSE: 0.4921
Epoch 6, Train Loss: 0.2357, Test RMSE: 0.4897
Epoch 7, Train Loss: 0.2303, Test RMSE: 0.4866
Epoch 8, Train Loss: 0.2233, Test RMSE: 0.4828
Epoch 9, Train Loss: 0.2145, Test RMSE: 0.4781
Epoch 10, Train Loss: 0.2039, Test RMSE: 0.4722


In [ ]:
print("Train matrix shape:", train_matrix.shape)
print("Test matrix shape:", test_matrix.shape)
print("Sample train matrix row (first user):", train_matrix[0])

Train matrix shape: torch.Size([610, 9724])
Test matrix shape: torch.Size([610, 9724])
Sample train matrix row (first user): tensor([0.8000, 0.8000, 0.8000,  ..., 0.0000, 0.0000, 0.0000])
